In [1]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
import time

from betfair import Betfair

pd.set_option('display.max_columns', None)

In [2]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [3]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

<KeepAliveResource>

In [4]:
# Set the start and end time to cover the entire day in UTC
today = datetime.utcnow()
start_time = today.replace(hour=0, minute=0, second=0, microsecond=0)
end_time = today.replace(hour=23, minute=59, second=59, microsecond=999999)

In [5]:
# Define filter for horse racing markets
market_filter = betfairlightweight.filters.market_filter(
    event_type_ids=['7'],
    market_countries=['GB'],
    market_type_codes=['WIN'], 
    market_start_time={'from': start_time.isoformat(),
                       'to' : end_time.isoformat()}
)

# Request market catalogue for all horse races on the date
results = trading.betting.list_market_catalogue(
    filter=market_filter,
    market_projection=["RUNNER_DESCRIPTION", 
            "RUNNER_METADATA", 
            "COMPETITION", 
            "EVENT", 
            "EVENT_TYPE", 
            "MARKET_DESCRIPTION", 
            "MARKET_START_TIME",],  # Include runner metadata
    max_results=1000  # Adjust based on expected number of races
)

In [6]:
for i in results:
        print(
            "{0} {1:d}:{2:02d} {3} ({4})".format(
                i.market_id,
                i.market_start_time.hour, 
                i.market_start_time.minute, 
                i.event.venue,  
                i.description.market_type,
            )
        )


1.232091511 12:35 Newmarket (WIN)
1.232092785 12:50 York (WIN)
1.232091550 13:00 Ffos Las (WIN)
1.232091517 13:10 Newmarket (WIN)
1.232092792 13:25 York (WIN)
1.232091557 13:35 Ffos Las (WIN)
1.232091524 13:45 Newmarket (WIN)
1.232092799 14:00 York (WIN)
1.232091719 14:10 Ffos Las (WIN)
1.232091531 14:20 Newmarket (WIN)
1.232092806 14:35 York (WIN)
1.232091786 14:45 Ffos Las (WIN)
1.232091537 14:55 Newmarket (WIN)
1.232092813 15:10 York (WIN)
1.232091799 15:20 Ffos Las (WIN)
1.232091542 15:30 Newmarket (WIN)
1.232092831 15:45 York (WIN)
1.232091792 15:55 Ffos Las (WIN)
1.232091806 16:15 Goodwood (WIN)
1.232092822 16:20 York (WIN)
1.232091845 16:35 Hamilton (WIN)
1.232091813 16:50 Goodwood (WIN)
1.232091852 17:05 Hamilton (WIN)
1.232091820 17:20 Goodwood (WIN)
1.232091859 17:35 Hamilton (WIN)
1.232091826 17:50 Goodwood (WIN)
1.232091866 18:05 Hamilton (WIN)
1.232091832 18:20 Goodwood (WIN)
1.232091873 18:35 Hamilton (WIN)
1.232091839 18:54 Goodwood (WIN)
1.232091880 19:09 Hamilton (WIN)

In [7]:
import pandas as pd

# Initialize an empty list to store market and runner details
data = []

# Iterate over each market in the market catalogue
for market in results:
    market_id = market.market_id
    market_name = market.market_name
    market_start_time = market.market_start_time
    market_type = market.description.market_type

    # Extract the location/venue of the race
    event = market.event
    race_location = event.venue if event else "Unknown Location"
    
    # Iterate over each runner in the market
    for runner in market.runners:
        runner_id = runner.selection_id
        runner_name = runner.runner_name
        
        # Append a dictionary with all relevant data to the list
        data.append({
            'market_id': market_id,
            'market_start_time': market_start_time,
            'venue': race_location,
            'market_type': market_type,
            'selection_id': runner_id,
            'horse_name': runner_name,
        })

# Convert the list of dictionaries to a pandas DataFrame
df1 = pd.DataFrame(data)

# Display the DataFrame
print(df1)


       market_id   market_start_time      venue market_type  selection_id  \
0    1.232091511 2024-08-23 12:35:00  Newmarket         WIN      10371597   
1    1.232091511 2024-08-23 12:35:00  Newmarket         WIN      24711668   
2    1.232091511 2024-08-23 12:35:00  Newmarket         WIN      72923505   
3    1.232091511 2024-08-23 12:35:00  Newmarket         WIN      72923502   
4    1.232091511 2024-08-23 12:35:00  Newmarket         WIN      72923499   
..           ...                 ...        ...         ...           ...   
290  1.232091880 2024-08-23 19:09:00   Hamilton         WIN      45527635   
291  1.232091880 2024-08-23 19:09:00   Hamilton         WIN      56296857   
292  1.232091880 2024-08-23 19:09:00   Hamilton         WIN      54482335   
293  1.232091880 2024-08-23 19:09:00   Hamilton         WIN      45929505   
294  1.232091880 2024-08-23 19:09:00   Hamilton         WIN      45929239   

          horse_name  
0      Magical Trail  
1            Beronia  
2    K

# I need to get the a list of id's from the market_id column and then create a loop for the list_market_book 

In [8]:
market_id_list = df1['market_id'].drop_duplicates().to_list()
len(market_id_list)

31

In [9]:
# Define a delay between requests
delay = 0.05  # Delay in seconds

# Loop through market IDs and fetch data with a delay
market_books = []
for market_id in market_id_list:
    market_book = trading.betting.list_market_book(
        market_ids=[market_id],
        price_projection=betfairlightweight.filters.price_projection(
            price_data=betfairlightweight.filters.price_data(ex_all_offers=True)
        ),
    )
    market_books.append(market_book)
    
    # Print or process the market_book as needed
    print(market_book)
    
    # Sleep to avoid overloading the API
    time.sleep(delay)

[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]


In [10]:
# Example: if market_books is a list of lists of MarketBook objects
data = []

# Iterate over each list of MarketBook objects
for book_list in market_books:
    # Iterate over each MarketBook object in the current list
    for market_book in book_list:
        market_id = market_book.market_id
        status = market_book.status
        total_matched = market_book.total_matched

        # Iterate over each runner in the market
        for runner in market_book.runners:
            runner_id = runner.selection_id
            runner_status = runner.status
            last_price_traded = runner.last_price_traded
            runner_total_matched = runner.total_matched
            
            # Append a dictionary with all relevant data to the list
            data.append({
                'market_id': market_id,
                'status': status,
                'total_matched': total_matched,
                'selection_id': runner_id,
                'status': runner_status,
                'morning_price': last_price_traded,
            })



# Convert the list of dictionaries into a DataFrame
df2 = pd.DataFrame(data)

df2 = df2[['market_id', 'selection_id', 'status', 'morning_price', 'total_matched']]
df2

,market_id,selection_id,status,morning_price,total_matched
0,1.232091511,10371597,ACTIVE,4.4,4485.12
1,1.232091511,24711668,ACTIVE,4.4,4485.12
2,1.232091511,72923505,ACTIVE,6.2,4485.12
3,1.232091511,72923502,ACTIVE,6.2,4485.12
4,1.232091511,72923499,ACTIVE,8.8,4485.12
...,...,...,...,...,...
290,1.232091880,45527635,ACTIVE,6.8,1973.36
291,1.232091880,56296857,ACTIVE,9.6,1973.36
292,1.232091880,54482335,ACTIVE,10.0,1973.36
293,1.232091880,45929505,ACTIVE,13.0,1973.36


# join df1 and df2

In [11]:
df3 = df1.merge(df2, on = ['market_id', 'selection_id'], how = 'left')

df3['market_id'] = df3['market_id'].astype(str)


current_date = datetime.now().strftime('%Y-%m-%d')
file_name = f"{current_date}_data.csv"


df3.to_csv('../data/daily/' + file_name, index=False)
df3

,market_id,market_start_time,venue,market_type,selection_id,horse_name,status,morning_price,total_matched
0,1.232091511,2024-08-23 12:35:00,Newmarket,WIN,10371597,Magical Trail,ACTIVE,4.4,4485.12
1,1.232091511,2024-08-23 12:35:00,Newmarket,WIN,24711668,Beronia,ACTIVE,4.4,4485.12
2,1.232091511,2024-08-23 12:35:00,Newmarket,WIN,72923505,Kathleens Glory,ACTIVE,6.2,4485.12
3,1.232091511,2024-08-23 12:35:00,Newmarket,WIN,72923502,Jowddah,ACTIVE,6.2,4485.12
4,1.232091511,2024-08-23 12:35:00,Newmarket,WIN,72923499,Bintjeddah,ACTIVE,8.8,4485.12
...,...,...,...,...,...,...,...,...,...
290,1.232091880,2024-08-23 19:09:00,Hamilton,WIN,45527635,Wee Fat Mac,ACTIVE,6.8,1973.36
291,1.232091880,2024-08-23 19:09:00,Hamilton,WIN,56296857,Lady Bouquet,ACTIVE,9.6,1973.36
292,1.232091880,2024-08-23 19:09:00,Hamilton,WIN,54482335,Zaphea,ACTIVE,10.0,1973.36
293,1.232091880,2024-08-23 19:09:00,Hamilton,WIN,45929505,Jackie Brown,ACTIVE,13.0,1973.36
